In [1]:
import tensorflow as tf
import numpy as np

/Users/andyxie/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
tf.__version__

'1.4.0'

# 开发环境
首先请根据官网的教程安装TensorFlow，本教程是根据1.4.0版本制作的，如果你是用其他的版本，因为机器学习最近的火热，导致API不断变化，很可能会遇到很多问题。

如果你还没有安装，那么建议先安装Anaconda。然后仅安装CPU版本的TensorFlow。GPU版本的安装非常麻烦，而且有许多硬件要求。在初级教程阶段，计算量比较小。具体指令如下：

```bash
pip3 install tensorflow     # Python 3.n; CPU support
```

# 为什么要使用TensorFlow
因为游戏行业的火热，经过数十年持续投资，近年GPU已经能够以非常快的速度进行矩阵运算。这个突破也成为了机器学习快速发展的契机。为了能够利用GPU的运算速度，我们需要使用专业工具。TensorFlow是当前流行的机器学习平台，它的远离是将你的代码编译成一系列CPU和GPU需要执行的指令，这样就不需要在运算过程中运行python代码，对于数据进行反复存储，或在GPU和内存间反复切换。通过只用TF可以大幅增加代码的运算速度。

# 基础操作

## 常量、变量和运算
TF的基本单位成为Tensor（张量），可以将其理解为高维数组，或者矩阵。Tensor间可以进行我们在线性代数中学习到的各种运算，但是必须要使用TF提供的函数，这样在运行过程中TF才能将其编译。

In [37]:
node1 = tf.constant(3.0, dtype=tf.float32) #常量
node2 = tf.constant(4.0) # 类型可以不声明
node1, node2 # 在声明后，所有的常量还只是object

(<tf.Tensor 'Const_4:0' shape=() dtype=float32>,
 <tf.Tensor 'Const_5:0' shape=() dtype=float32>)

In [38]:
sess = tf.Session()
sess.run([node1, node2]) #只有run之后，常量才会变成值

[3.0, 4.0]

In [39]:
node3 = tf.add(node1, node2) #tf中有加减乘除等各种运算的函数
node3

<tf.Tensor 'Add_3:0' shape=() dtype=float32>

In [40]:
sess.run(node3)

7.0

当然，相较于常量，我们更经常使用变量。在进行学习的时候，所有的参数都应该使用变量进行定义。它的使用方法和常量没有什么区别。唯一需要注意的是使用前需要初始化。

In [66]:
k = tf.Variable([-3], dtype=tf.float32)
init = tf.global_variables_initializer()
sess.run(init)
sess.run(k)

array([-3.], dtype=float32)

## 未知量
在运算当中，对于x，y等未知的量，tf使用placeholder（占位符）表示。

In [50]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = tf.add(a,b)
c

<tf.Tensor 'Add_5:0' shape=<unknown> dtype=float32>

In [51]:
sess.run(c, {a: 3, b: 4}) # 对a，b赋值，以计算c

7.0

In [52]:
sess.run(c, {a:[1,2,3], b:[4,5,6]}) # 分别给a，b数个值，则进行矩阵计算，输出数组

array([ 5.,  7.,  9.], dtype=float32)

# 例子：线性回归
问题：假设我们有下列x，y值，请建立一个线性模型：
x: [1, 2, 3, 4], y: [0, -1, -2, -3]

讨论：观察数据，不难得到 y = -x + 1 这个模型。那么怎样使用TF求出来呢？

In [53]:
# 首先，建立一个线性模型
y = tf.placeholder(tf.float32) # 在模型中Y是未知量，我们将使用题目中的y值填充y，以求得其他参数
x = tf.placeholder(tf.float32) # 同上
w = tf.Variable([0.3], dtype=tf.float32)
b = tf.Variable([-0.3], dtype=tf.float32) #0.01只是初始化的值，相较于0可以提高计算速度
y_ = w * x + b # TF已经对运算符进行了重载
y_

<tf.Tensor 'add_6:0' shape=<unknown> dtype=float32>

In [60]:
# 任何模型都有一个loss（成本），loss标志着模型的准确度，通过不断减少loss，最后得到的参数将具有预测能力
loss = tf.squared_difference(y_, y) #loss = (y_ - y)^2
init = tf.global_variables_initializer()
sess.run(init)
sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

array([  0.        ,   1.68999982,   6.75999928,  15.21000099], dtype=float32)

In [55]:
# 通过使用梯度下降，我们将通过修改函数，不断修正参数的值
optimizer = tf.train.GradientDescentOptimizer(0.01) #0.01是learning_rate(学习比率)
train = optimizer.minimize(loss)

In [67]:
# 重复梯度下降1000次
for i in range(3000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

sess.run([w, b]) # 计算w，b

[array([-0.99999911], dtype=float32), array([ 0.99999744], dtype=float32)]

In [68]:
# 使用模型可以得到如下计算值
sess.run(y_, {x: [1, 2, 3, 4]})

array([ -1.66893005e-06,  -1.00000072e+00,  -2.00000000e+00,
        -2.99999905e+00], dtype=float32)